In [1]:
import sys
#sys.path.append('../shared/')
#import videodisko as vd
# Imports
import numpy as np
import numpy.matlib
import numpy.random
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import axes3d 

from matplotlib import pyplot as plt

from scipy import ndimage
from scipy import misc
from scipy import fftpack
from scipy import optimize
import scipy.io
import skimage
import imageio
import colour as c
import math
import csv

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import pandas as pd

import cv2
import time

from datetime import datetime

import os
from fnmatch import fnmatch

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import os
from fnmatch import fnmatch
import os
from fnmatch import fnmatch
from matplotlib.patches import Rectangle
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap

In [2]:
import sys
#sys.path.append('../shared/')
#import videodisko as vd
# Imports
import numpy as np
import numpy.matlib
import numpy.random
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import axes3d 

from matplotlib import pyplot as plt

from scipy import ndimage
from scipy import misc
from scipy import fftpack
from scipy import optimize
import scipy.io
import skimage
import imageio
import colour as c
import math
import csv

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import pandas as pd

import cv2
import time

from datetime import datetime

import os
from fnmatch import fnmatch




## Linear to sRGB
def lin2srgb (x):
    result = (x <= 0.0031308) * np.multiply(x,12.92) + (x > 0.0031308) * np.subtract(np.multiply(1.055,np.power(x,(1/2.4))),0.055)
    return result


## sRGB to Linear 
def srgb2lin (x):
    result = (x <= 0.04045) * np.divide(x,12.92) + (x > 0.04045) * np.power(np.divide(np.add(x,0.055),1.055),2.4)
    return result

# SDR Formulas

## Input Luminance between 0-1
## Output Videosignal
def oetf709(x): 
    return np.subtract(np.multiply(np.power(x, 0.45), 1.099), 0.099)*(0.018<= x) + np.multiply(4.5, x) * (x <0.018)

def eotf1886(x, Lw = 100, Lb = 0): 
    gamma = 2.4
    a = np.power(np.subtract(np.power(Lw, np.divide(1,gamma)),np.power(Lb, np.divide(1, gamma))), gamma)
    b = np.divide(np.power(Lb, np.divide(1,gamma)) , np.subtract(np.power(Lw , np.divide(1,gamma)), np.power( Lb , np.divide(1, gamma))))
    return np.multiply(a , np.power( np.maximum(np.add(x, b), 0) , gamma))

def eotf1886_CRT(x, Lw = 100, Lb = 0, b = 0): 
    Vc = 0.35
    a1 = 2.6
    a2 = 3.0
    k = np.subtract(Lw,np.power(np.add(1,b),a1))
    return (x < Vc) * np.multiply(np.multiply(k,np.power(np.add(Vc,b),np.subtract(a1,a2))),np.power(np.add(x,b),a2)) + (Vc <= x) * np.multiply(k,np.power(np.add(x,b),a1))

## Luma Formulas for Rec2020 / Rec709 -> Fixed Addition
def luma709(img): 
    return np.multiply(img[:,:,0],0.2126) + np.multiply(img[:,:,1],0.7152) + np.multiply(img[:,:,2],0.0722)
def luma2020(img): 
    return np.multiply(img[:,:,0],0.2627) + np.multiply(img[:,:,1],0.6780) + np.multiply(img[:,:,2],0.0593)

## Formulas for PQ

# increasing rec.709 formulas 
def steroids709(x): 
    return np.subtract(np.multiply(np.power(np.multiply(59.5208 , x), 0.45), 1.099), 0.099)*(0.0003024< x) + np.multiply(267.84, x) * (x <= 0.0003024)

def steroids1886(x): 
    return np.multiply(100, np.power(x, 2.4))


## Transportfunctions

def pq_eotf(x): 
    ##m1
    m1 = np.divide(2610, 16384) ## 2^14 = 16384
    m2 = np.multiply(np.divide(2523, 4096), 128) ##2^12 = 4096 , 2^7 = 128 -> Jan fragen
    c1 = np.divide(3424, 4096)
    c2 = np.multiply(np.divide(2413, 4096), 32)
    c3 = np.multiply(np.divide(2392, 4096), 32)

    
    return np.multiply(np.power(np.divide(np.maximum( np.subtract( np.power(x , np.divide(1 , m2)), c1) , 0) , np.subtract(c2, np.multiply(c3 , np.power(x , np.divide(1 , m2))))) , np.divide(1, m1)), 10000)

## Inverse_eotf

def pq_eotf_inv(x): 
    m1 = np.divide(2610, 16384) ## 2^14 = 16384
    m2 = np.multiply(np.divide(2523, 4096), 128) ##2^12 = 4096 , 2^7 = 128 -> Jan fragen
    c1 = np.divide(3424, 4096)
    c2 = np.multiply(np.divide(2413, 4096), 32)
    c3 = np.multiply(np.divide(2392, 4096), 32)

    y = np.divide(x , 10000) ## Normalisierung auf 0.00 bis 1.0

    return np.power( np.divide(np.add(c1 , np.multiply(c2 , np.power(y , m1))), np.add(1 , np.multiply(c3 , np.power(y , m1)))), m2)

# RGB Input
def hlg_oetf(x): 
    a = 0.17883277
    b = np.subtract(1 , np.multiply(4 , a))
    c = np.subtract(0.5 , np.multiply(a , np.log(np.multiply(4 , a))))

    return np.where(x <= np.divide(1,12), np.sqrt(np.multiply(3,x )), np.add( np.multiply(a , np.log(np.subtract( np.multiply(12 , x ), b))) , c) )

def hlg_oetf_inv(x):
    a = 0.17883277
    b = np.subtract(1 , np.multiply(4 , a))
    c = np.subtract(0.5 , np.multiply(a , np.log(np.multiply(4 , a))))
    
    return np.where(x <= 0.5 , np.divide( np.power(x,2), 3), np.divide(np.add(np.exp(np.divide(np.subtract(x , c), a)), b) , 12) )

## Lw = Usergain, nominal peak Luminance of the display (achromatic)
def hlg_ootf(x, Lw): 
    r = x[: , : , 0]
    g = x[: , : , 1]
    b = x[: , : , 2]
    luma = np.add(np.multiply(0.2627 ,r ), np.multiply(0.678 , g), np.multiply(0.0593 , b))
    alpha = Lw
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    rd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), r)
    gd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), g)
    bd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), b)
    return np.stack((rd , gd , bd))

## Lw = Usergain, nominal peak Luminance of the display (achromatic)
def hlg_ootf_bw(x, Lw=1000): 
    alpha = Lw
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    return np.multiply(np.multiply( alpha , np.power(x , np.subtract(gamma , 1))),x)

def hlg_eotf(x, Lw = 1000, Lb = 0.005):
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    beta = np.sqrt(np.multiply(3 , np.power(np.divide(Lb , Lw), np.divide(1,gamma))))
    return hlg_ootf_bw(hlg_oetf_inv(np.maximum(0 , (np.add( np.multiply( np.subtract(1 , beta), x ), beta)))), Lw)


def slog3oetf(x): 
    # reflection add
    x = x*0.9

    return (x < 0.01125)* np.divide( np.add( np.multiply(x , np.divide(np.subtract(171.2102946929, 95), 0.01125)), 95), 1023) + (x >= 0.01125)* np.divide(np.add(420 , np.multiply( np.log10(np.divide(np.add(x,0.01), np.add(0.18, 0.01))), 261.5)), 1023)

def slog3eotf(x): 
    return (x >= np.divide(171.2102946929,1023)) * np.subtract( np.multiply(np.power(10 , np.divide( np.subtract(420 , np.multiply(x , 1023)) , 261.5)) , np.add(0.18 , 0.01)) , 0.01) + (x < np.divide(171.2102946929,1023)) * np.multiply(np.subtract(np.multiply(x , 1023), 95 ) , np.divide(0.01125 , np.subtract(171.2102946929 , 95)))

# These are the ARRI LogCv3 Formulas
def lin2logc(x):
    return np.multiply(x > 0.010591, 0.24719 * np.log10(5.555556 * x + 0.052272) + 0.385537) + np.multiply(x <= 0.010591, 5.367655 * x + 0.092809)
def logc2lin(x):
    return np.multiply(x > 0.149659, (np.power(10.0, (x - 0.385537) / 0.247189) - 0.052272) / 5.555556) + np.multiply(x <= 0.149659, (x - 0.092809) / 5.367650)


### LOG C4
def Encoding_LogC4(x):
    a = (np.power(2.0, 18.0) - 16.0) / 117.45
    b = (1023.0 - 95.0) / 1023.0
    c = 95.0 / 1023.0
    s = (7 * np.log(2) * np.power(2.0, 7 - 14 * c / b)) / (a * b)
    t = (np.power(2.0, 14.0 * (-c / b) + 6.0) - 64.0) / a
    return (x < t) * np.divide(np.subtract(x,t),s) + (x >= t) * np.add(np.multiply(np.divide( np.subtract(np.log2(np.add(np.multiply(a,x),64.0)),6.0),14.0),b),c)

def Decoding_LogC4(x):
    a = (np.power(2.0, 18.0) - 16.0) / 117.45
    b = (1023.0 - 95.0) / 1023.0
    c = 95.0 / 1023.0
    s = (7 * np.log(2) * np.power(2.0, 7 - 14 * c / b)) / (a * b)
    t = (np.power(2.0, 14.0 * (-c / b) + 6.0) - 64.0) / a
    return (x < 0.0) * np.add(np.multiply(x,s),t) + (x >= 0.0) * np.divide(np.subtract(np.power(2.0, np.add(np.multiply(14,np.divide(np.subtract(x,c),b)),6)),64.0),a)


def full2legal(x):
    return (219*(x)+16)*np.power(2,(10-8))

def legal2full(x):
    return (((x/np.power(2,(10-8)))-16)/219)

def lin2log2(x):
    return np.log2(x)

def log22lin(x):
    return np.power(2,x)

def Blenden_Differenz(num1, num2):
    new2 = num2 / num1
    return np.log2(new2)

def mse_calculation(imageA, imageB):
    # Expects Luma Images
	err = np.mean(np.square(np.subtract(imageA,imageB)))
	return err




def BT2020_Linear_to_ICtCp(original):
    ictcp_array = np.zeros((np.shape(original)))

    L1 = (1688*original[:,:,0] + 2146*original[:,:,1] + 262*original[:,:,2])/4096
    M1 = (683*original[:,:,0] + 2951*original[:,:,1] + 462*original[:,:,2])/4096
    S1 = (99*original[:,:,0] + 309*original[:,:,1] + 3688*original[:,:,2])/4096

    L1_Mapped = c.models.eotf_inverse_ST2084(L1)
    M1_Mapped = c.models.eotf_inverse_ST2084(M1)
    S1_Mapped = c.models.eotf_inverse_ST2084(S1)

    ictcp_array[:,:,0] = 0.5*L1_Mapped+0.5*M1_Mapped
    ictcp_array[:,:,1] =(6610*L1_Mapped - 13613*M1_Mapped + 7003*S1_Mapped)/4096
    ictcp_array[:,:,2] =(17933*L1_Mapped - 17390*M1_Mapped - 543*S1_Mapped)/4096

    return ictcp_array






import numpy as np

def flatten_and_reshape_array(original_array, new_shape):
    flattened_array = np.array(original_array).flatten()
    reshaped_array = flattened_array[:np.prod(new_shape)].reshape(new_shape)
    return reshaped_array

# Create a sample 2D array
sample_2d_array = np.random.randint(1, 10, size=(8, 10))

# Specify the new shape (for example, 4x20)
new_shape = (1080, 1920)






import numpy as np

def create_dict_from_arrays(array1, array2):
    # Ensure both arrays have the same size
    assert array1.shape == array2.shape, "Arrays must have the same size"

    # Create a dictionary using a dictionary comprehension
    result_dict = {array1[i, j]: array2[i, j] for i in range(array1.shape[0]) for j in range(array1.shape[1])}

    return result_dict

# # Example arrays (replace with your actual arrays)
# array1 = np.array([[1, 2, 3], [4, 5, 6]])
# array2 = np.array([[10, 20, 30], [40, 50, 60]])





import csv

def read_csv_into_2d_array(file_path):
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        # Use list comprehension to create a 2D array from the CSV rows
        csv_data_2d_array = [row for row in csvreader]

    return csv_data_2d_array





def remove_items_below_threshold(dictionary, threshold):
    # Use a dictionary comprehension to filter key-value pairs based on the threshold
    filtered_dict = {key: value for key, value in dictionary.items() if value >= threshold}

    return filtered_dict




import heapq

def get_top_n_items(dictionary, n):
    # Use heapq.nlargest to get the top n key-value pairs based on values
    top_n_items = heapq.nlargest(n, dictionary.items(), key=lambda item: item[1])

    # Convert the result back to a dictionary
    top_n_dict = dict(top_n_items)

    return top_n_dict

# Example usage:
my_dict = {'key1': 1.2, 'key2': 2.5, 'key3': 3.0, 'key4': 4.2, 'key5': 5.5, 'key6': 6.0}
top_n = 3  # Change this to 20000 for your specific case

top_n_dict = get_top_n_items(my_dict, top_n)

print("Original Dictionary:", my_dict)
print(f"Top {top_n} Dictionary based on values:", top_n_dict)







Original Dictionary: {'key1': 1.2, 'key2': 2.5, 'key3': 3.0, 'key4': 4.2, 'key5': 5.5, 'key6': 6.0}
Top 3 Dictionary based on values: {'key6': 6.0, 'key5': 5.5, 'key4': 4.2}


In [3]:
import shutil
import os

def delete_folder_recursive(root_directory, folder_to_delete):
    for root, dirs, files in os.walk(root_directory, topdown=False):
        for current_dir in dirs:
            if current_dir == folder_to_delete:
                folder_path = os.path.join(root, current_dir)
                try:
                    shutil.rmtree(folder_path)
                    print(f"Folder '{folder_path}' and its contents successfully deleted.")
                except OSError as e:
                    print(f"Error: {e}")

# Specify the path to the root directory where you want to search and delete the folder
root_directory = '/Volumes/JM_SSD/61.2_Auswertung_Matlab_Neu'

# Specify the folder name you want to delete
folder_to_delete = '5_F002C012_220412_R07V'

# Call the function to delete the folder recursively
delete_folder_recursive(root_directory, folder_to_delete)


Folder '/Volumes/JM_SSD/61.2_Auswertung_Matlab_Neu/HLG_HD_vs_HLG_FR/02_ProRes_422HQ/12.1_TRANSFORMED_HLG_FR_UHD/ARRI_Encounters/5_F002C012_220412_R07V' and its contents successfully deleted.
Folder '/Volumes/JM_SSD/61.2_Auswertung_Matlab_Neu/HLG_HD_vs_HLG_FR/09_DNxHD_365/12.1_TRANSFORMED_HLG_FR_UHD/ARRI_Encounters/5_F002C012_220412_R07V' and its contents successfully deleted.
Folder '/Volumes/JM_SSD/61.2_Auswertung_Matlab_Neu/HLG_vs_PQ/02_ProRes_422HQ/11.1_TRANSFORMED_PQ_FR_UHD/ARRI_Encounters/5_F002C012_220412_R07V' and its contents successfully deleted.
Folder '/Volumes/JM_SSD/61.2_Auswertung_Matlab_Neu/HLG_vs_PQ/03.2_JPEG_2000_Level_4/11.1_TRANSFORMED_PQ_FR_UHD/ARRI_Encounters/5_F002C012_220412_R07V' and its contents successfully deleted.
Folder '/Volumes/JM_SSD/61.2_Auswertung_Matlab_Neu/HLG_vs_PQ/04_XAVC_Class_100/11.1_TRANSFORMED_PQ_FR_UHD/ARRI_Encounters/5_F002C012_220412_R07V' and its contents successfully deleted.
Folder '/Volumes/JM_SSD/61.2_Auswertung_Matlab_Neu/HLG_vs_PQ/05

In [101]:
import numpy as np

# Sample arrays
array1 = np.array(c.models.eotf_HLG_BT2100(legal2full(np.linspace(64,940,877)),0.005,1000))
array2 = np.array(np.clip(c.models.eotf_ST2084(legal2full(np.linspace(78,722,645))),0.005,1000))

# Create a new array
new_array = []

# Iterate through each element in array1
for value in array1:
    # Find the closest value in array2
    closest_value = min(array2, key=lambda x: abs(x - value))
    
    # Append a pair [value_from_array1, closest_value_from_array2] to the new array
    new_array.append([value, closest_value])

# Convert the new array to a NumPy array
new_array_HLG = np.array(new_array)

print(new_array)


[[0.0049999999999999992, 0.0055938061006096211], [0.0063600138789661519, 0.0063938605542215032], [0.007914423792992991, 0.0081702404049783066], [0.0096704508576944768, 0.01018996717705101], [0.011634947263367055, 0.011294311088322583], [0.013814442674438035, 0.013698634410638604], [0.016215181823826696, 0.016372979320192559], [0.01884315546125297, 0.019328667293666751], [0.021704126180476645, 0.020915647321149398], [0.024803650229695558, 0.024315328582558492], [0.028147096122026979, 0.028026146140639759], [0.031739660661892329, 0.032060599062800739], [0.035586382859271619, 0.036431517331512009], [0.039692156098895828, 0.038747251223081848], [0.044061738853640453, 0.043647652206005215], [0.048699764172779859, 0.048918055800086568], [0.053610748131010272, 0.054572471765236268], [0.058799097389545057, 0.057548157239613706], [0.064269115993519935, 0.063805621261669443], [0.070025011508557444, 0.070483654007394952], [0.076070900582272311, 0.077597506722840909], [0.082410814002762456, 0.0813

/Users/justus/.local/lib/python3.9/site-packages/colour/utilities/verbose.py:262: ColourUsageWarning: "Recommendation ITU-R BT.2100" "Reference HLG OOTF" uses RGB Luminance in computations and expects a vector input, thus the given input array will be stacked to compose a vector for internal computations but a single component will be output.
  warn(*args, **kwargs)


In [102]:
import numpy as np

# Sample arrays
array2 = np.array(c.models.eotf_HLG_BT2100(legal2full(np.linspace(64,940,877)),0.005,1000))
array1 = np.array(c.models.eotf_ST2084(legal2full(np.linspace(78,722,645))))

# Create a new array
new_array = []

# Iterate through each element in array1
for value in array1:
    # Find the closest value in array2
    closest_value = min(array2, key=lambda x: abs(x - value))
    
    # Append a pair [value_from_array1, closest_value_from_array2] to the new array
    new_array.append([value, closest_value])

# Convert the new array to a NumPy array
new_array_PQ = np.array(new_array)

print(new_array)


[[0.0055938061006096211, 0.0049999999999999992], [0.0063938605542215032, 0.0063600138789661519], [0.0072522635156068802, 0.007914423792992991], [0.0081702404049783066, 0.007914423792992991], [0.0091490470055897478, 0.0096704508576944768], [0.01018996717705101, 0.0096704508576944768], [0.011294311088322583, 0.011634947263367055], [0.012463413843741885, 0.011634947263367055], [0.013698634410638604, 0.013814442674438035], [0.015001354781431213, 0.013814442674438035], [0.016372979320192559, 0.016215181823826696], [0.017814934255923419, 0.01884315546125297], [0.019328667293666751, 0.01884315546125297], [0.020915647321149398, 0.021704126180476645], [0.022577364193536009, 0.021704126180476645], [0.024315328582558492, 0.024803650229695558], [0.026131071879122128, 0.024803650229695558], [0.028026146140639759, 0.028147096122026979], [0.030002124076051656, 0.031739660661892329], [0.032060599062800739, 0.031739660661892329], [0.034203185191078476, 0.035586382859271619], [0.036431517331512009, 0.03

In [31]:
c.models.eotf_ST2084(legal2full(np.linspace(70,722,659)))

array([  0.00000000e+00,   5.25912035e-05,   1.70826479e-04,
         3.42874260e-04,   5.65730978e-04,   8.38361594e-04,
         1.16057086e-03,   1.53261170e-03,   1.95500928e-03,
         2.42846921e-03,   2.95382485e-03,   3.53200479e-03,
         4.16401172e-03,   4.85090808e-03,   5.59380610e-03,
         6.39386055e-03,   7.25226352e-03,   8.17024040e-03,
         9.14904701e-03,   1.01899672e-02,   1.12943111e-02,
         1.24634138e-02,   1.36986344e-02,   1.50013548e-02,
         1.63729793e-02,   1.78149343e-02,   1.93286673e-02,
         2.09156473e-02,   2.25773642e-02,   2.43153286e-02,
         2.61310719e-02,   2.80261461e-02,   3.00021241e-02,
         3.20605991e-02,   3.42031852e-02,   3.64315173e-02,
         3.87472512e-02,   4.11520636e-02,   4.36476522e-02,
         4.62357361e-02,   4.89180558e-02,   5.16963731e-02,
         5.45724718e-02,   5.75481572e-02,   6.06252571e-02,
         6.38056213e-02,   6.70911219e-02,   7.04836540e-02,
         7.39851353e-02,

In [63]:
np.linspace(64,940,877)

array([  64.,   65.,   66.,   67.,   68.,   69.,   70.,   71.,   72.,
         73.,   74.,   75.,   76.,   77.,   78.,   79.,   80.,   81.,
         82.,   83.,   84.,   85.,   86.,   87.,   88.,   89.,   90.,
         91.,   92.,   93.,   94.,   95.,   96.,   97.,   98.,   99.,
        100.,  101.,  102.,  103.,  104.,  105.,  106.,  107.,  108.,
        109.,  110.,  111.,  112.,  113.,  114.,  115.,  116.,  117.,
        118.,  119.,  120.,  121.,  122.,  123.,  124.,  125.,  126.,
        127.,  128.,  129.,  130.,  131.,  132.,  133.,  134.,  135.,
        136.,  137.,  138.,  139.,  140.,  141.,  142.,  143.,  144.,
        145.,  146.,  147.,  148.,  149.,  150.,  151.,  152.,  153.,
        154.,  155.,  156.,  157.,  158.,  159.,  160.,  161.,  162.,
        163.,  164.,  165.,  166.,  167.,  168.,  169.,  170.,  171.,
        172.,  173.,  174.,  175.,  176.,  177.,  178.,  179.,  180.,
        181.,  182.,  183.,  184.,  185.,  186.,  187.,  188.,  189.,
        190.,  191.,

In [62]:
c.models.eotf_ST2084(legal2full(722))

993.73075915502545

In [103]:
new_test_array_HLG = np.zeros((2160,3840,3))

new_test_array_HLG[0,0:877,0] = c.models.eotf_inverse_HLG_BT2100(new_array_HLG[:,0],0.005,1000)
new_test_array_HLG[0,0:877,1] = c.models.eotf_inverse_HLG_BT2100(new_array_HLG[:,0],0.005,1000)
new_test_array_HLG[0,0:877,2] = c.models.eotf_inverse_HLG_BT2100(new_array_HLG[:,0],0.005,1000)

/Users/justus/.local/lib/python3.9/site-packages/colour/models/rgb/transfer_functions/arib_std_b67.py:115: RuntimeWarning: invalid value encountered in log
  E <= 1, r * gamma_function(E, 0.5, "mirror"), a * np.log(E - b) + c


In [104]:
new_test_array_PQ = np.zeros((2160,3840,3))

new_test_array_PQ[0,0:877,0] = c.models.eotf_inverse_ST2084(new_array_HLG[:,1])
new_test_array_PQ[0,0:877,1] = c.models.eotf_inverse_ST2084(new_array_HLG[:,1])
new_test_array_PQ[0,0:877,2] = c.models.eotf_inverse_ST2084(new_array_HLG[:,1])

In [105]:
new_test_array_HLG1 = np.zeros((2160,3840,3))

new_test_array_HLG1[0,0:645,0] = c.models.eotf_inverse_HLG_BT2100(new_array_PQ[:,0],0.005,1000)
new_test_array_HLG1[0,0:645,1] = c.models.eotf_inverse_HLG_BT2100(new_array_PQ[:,0],0.005,1000)
new_test_array_HLG1[0,0:645,2] = c.models.eotf_inverse_HLG_BT2100(new_array_PQ[:,0],0.005,1000)

In [106]:
new_test_array_PQ1 = np.zeros((2160,3840,3))

new_test_array_PQ1[0,0:645,0] = c.models.eotf_inverse_ST2084(new_array_PQ[:,1])
new_test_array_PQ1[0,0:645,1] = c.models.eotf_inverse_ST2084(new_array_PQ[:,1])
new_test_array_PQ1[0,0:645,2] = c.models.eotf_inverse_ST2084(new_array_PQ[:,1])

In [28]:
new_test_array_PQ[100,1000,0]

0.0

In [110]:
new_test_array_HLG[1:,:, :] = new_test_array_HLG[0,:,:]

In [111]:
new_test_array_HLG1[1:,:, :] = new_test_array_HLG1[0,:,:]

In [112]:
new_test_array_PQ[1:,:, :] = new_test_array_PQ[0,:,:]

In [113]:
new_test_array_PQ1[1:,:, :] = new_test_array_PQ1[0,:,:]

In [15]:
new_test_array_HLG[100,200,:]

array([ 0.23657546,  0.23657546,  0.23657546])

In [116]:
new_test_array_PQ[200,2000,:]

array([ 0.,  0.,  0.])

In [73]:
new_test_array_HLG[200,:,0] = 0
new_test_array_HLG[200,:,1] = 0
new_test_array_HLG[200,:,2] = 0

In [74]:
new_test_array_HLG[200:,:, :] = new_test_array_HLG[0,:,:]

In [100]:
new_test_array_HLG = np.zeros((1080,1920,3))
new_test_array_HLG[0,0:len(new_array_PQ),0] = new_array_PQ[:,0]
new_test_array_HLG[0,0:len(new_array_PQ),1] = new_array_PQ[:,0]
new_test_array_HLG[0,0:len(new_array_PQ),2] = new_array_PQ[:,0]

In [101]:
new_test_array_HLG[0:,:, :] = new_test_array_HLG[0,:,:]

In [83]:
new_test_array_HLG[0,:,0] = np.zeros((1,1920))
new_test_array_HLG[0,:,1] = np.zeros((1,1920))
new_test_array_HLG[0,:,2] = np.zeros((1,1920))

In [84]:
new_test_array_HLG[0:,:, :] = new_test_array_HLG[0,:,:]

In [92]:
c.write_image(new_test_array_HLG,"/Users/justus/Desktop/TEST_IMAGE_HLG_GRAY_SCALE_IN_HLG.tiff",bit_depth="uint16")

/Users/justus/.local/lib/python3.9/site-packages/colour/utilities/verbose.py:262: ColourUsageWarning: "OpenImageIO" related API features are not available, switching to "Imageio"!
  warn(*args, **kwargs)


In [93]:
c.write_image(new_test_array_PQ,"/Users/justus/Desktop/TEST_IMAGE_HLG_GRAY_SCALE_IN_PQ.tiff",bit_depth="uint16")

In [94]:
c.write_image(new_test_array_HLG1,"/Users/justus/Desktop/TEST_IMAGE_PQ_GRAY_SCALE_IN_HLG.tiff",bit_depth="uint16")

In [95]:
c.write_image(new_test_array_PQ1,"/Users/justus/Desktop/TEST_IMAGE_PQ_GRAY_SCALE_IN_PQ.tiff",bit_depth="uint16")

In [117]:
test01 = c.read_image("/Users/justus/Desktop/TEST_IMAGE_HLG_GRAY_SCALE_IN_HLG.tiff")
test02 = c.read_image("/Users/justus/Desktop/TEST_IMAGE_HLG_GRAY_SCALE_IN_PQ.tiff")
test03 = c.read_image("/Users/justus/Desktop/TEST_IMAGE_PQ_GRAY_SCALE_IN_HLG.tiff")
test04 = c.read_image("/Users/justus/Desktop/TEST_IMAGE_PQ_GRAY_SCALE_IN_PQ.tiff")

/Users/justus/.local/lib/python3.9/site-packages/colour/utilities/verbose.py:262: ColourUsageWarning: "OpenImageIO" related API features are not available, switching to "Imageio"!
  warn(*args, **kwargs)


In [118]:
test01[:,877:,:] = 0
test02[:,877:,:] = 0
test03[:,645:,:] = 0
test04[:,645:,:] = 0

In [119]:
c.write_image(test01,"/Users/justus/Desktop/TEST_IMAGE_HLG_GRAY_SCALE_IN_HLG1.tiff",bit_depth="uint16")
c.write_image(test02,"/Users/justus/Desktop/TEST_IMAGE_HLG_GRAY_SCALE_IN_PQ1.tiff",bit_depth="uint16")
c.write_image(test03,"/Users/justus/Desktop/TEST_IMAGE_PQ_GRAY_SCALE_IN_HLG1.tiff",bit_depth="uint16")
c.write_image(test04,"/Users/justus/Desktop/TEST_IMAGE_PQ_GRAY_SCALE_IN_PQ1.tiff",bit_depth="uint16")

/Users/justus/.local/lib/python3.9/site-packages/colour/utilities/verbose.py:262: ColourUsageWarning: "OpenImageIO" related API features are not available, switching to "Imageio"!
  warn(*args, **kwargs)


In [17]:
new_array_pq = new_array

In [9]:
new_array_hlg = new_array

In [22]:
new_array_pq[:,0] = new_array[:,1]
new_array_pq[:,1] = new_array[:,0]

In [ ]:
value_counter_image_hlg_FR = c.models.eotf_HLG_BT2100(legal2full(np.linspace(64,940,877)),0.005,1000)
value_counter_image_pq_FR = c.models.eotf_ST2084(legal2full(np.linspace(64,722,659)))

In [25]:
new_array_pq[:, 0] *= 2
np.shape(new_array_pq)

(659, 2)

In [ ]:
new_array_pq_only1 = new_array_pq

In [26]:
new_array_hlg[:, 0] *= 2
np.shape(new_array_hlg)

(877, 2)

In [24]:
new_big_array = new_array_hlg.extend(new_array_pq)

AttributeError: 'numpy.ndarray' object has no attribute 'extend'